In [14]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets.karate import KarateClub

In [15]:
dataset = KarateClub()
data = dataset[0]
print(data)

split_idx = data.num_nodes // 2
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.train_mask[:split_idx] = 1
data.test_mask = ~data.train_mask

Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])


#### Defining a very simple single layer GCNConv

In [16]:
class SingleLayerGCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SingleLayerGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        return F.log_softmax(x, dim=1)

In [17]:
model = SingleLayerGCN(dataset.num_features, dataset.num_classes)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = F.nll_loss

# Train the model
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [18]:
model.eval()
_, pred = model(data.x, data.edge_index).max(dim=1)
correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.5294
